<a href="https://colab.research.google.com/github/Junseokee/Deep_Learning/blob/main/220920_KOBERT%EB%A1%9C_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab 환경설정

In [4]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install folium==0.2.1
!pip install -U --no-cache-dir gdown --pre

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 103.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 8.8 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595741 sha256=005b497cd2f79e64ca8c933e8b20e42842ed9b64e145fc6e84094ad960ca4e1c
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [5]:
# 깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-et59nnuj
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-et59nnuj
     |████████████████████████████████| 129 kB 7.5 MB/s 
     |████████████████████████████████| 54.7 MB 39 kB/s 
     |████████████████████████████████| 4.5 MB 48.2 MB/s 
     |████████████████████████████████| 1.2 MB 64.7 MB/s 
     |██████████████████████████████▎ | 834.1 MB 81.0 MB/s eta 0:00:01tcmalloc: large alloc 1147494400 bytes == 0x39a1e000 @  0x7f6a1ee2b615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp # http://freesearch.pe.kr/arcives/4963
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [7]:
# kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
# huggingface로 kobert접근
# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from pretrained('skt/kobert-base-v1')
# model =AutoModel.from_pretrained('skt/kobert-base-v1')

In [9]:
#GPU 사용
device = torch.device('cuda:0')

In [10]:
# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


KoBERT vocab

In [11]:
!pip install -U --no-cache-dir gdown --prem


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --prem


In [12]:
print('Downloading 한국어_단발성_대화_데이터셋.xlsx...')
!gdown --id {'1EcVRC6UjL27w1lACQIVsIdFc89zPhb_C'} -O 한국어_단발성_대화_데이터셋.xlsx

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1EcVRC6UjL27w1lACQIVsIdFc89zPhb_C
To: /content/한국어_단발성_대화_데이터셋.xlsx
100% 1.64M/1.64M [00:00<00:00, 198MB/s]


In [13]:
import pandas as pd
data = pd.read_excel('/content/한국어_단발성_대화_데이터셋.xlsx')
data.head(7)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
5,남자친구가 떠날까봐요,공포,NaN,NaN,NaN,혐오,5429.0
6,이거 했는데 허리가 아플수도 있나요? ;;,공포,NaN,NaN,NaN,Total,38594.0


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38594 entries, 0 to 38593
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sentence    38594 non-null  object 
 1   Emotion     38594 non-null  object 
 2   Unnamed: 2  0 non-null      float64
 3   Unnamed: 3  0 non-null      float64
 4   Unnamed: 4  0 non-null      float64
 5   공포          7 non-null      object 
 6   5468        7 non-null      float64
dtypes: float64(4), object(3)
memory usage: 2.1+ MB


In [15]:
data.loc[(data['Emotion'] == '공포'), 'Emotion'] = 0 # 공포 => 0
data.loc[(data['Emotion'] == '놀람'), 'Emotion'] = 1 
data.loc[(data['Emotion'] == '분노'), 'Emotion'] = 2 
data.loc[(data['Emotion'] == '슬픔'), 'Emotion'] = 3 
data.loc[(data['Emotion'] == '중립'), 'Emotion'] = 4 
data.loc[(data['Emotion'] == '행복'), 'Emotion'] = 5 
data.loc[(data['Emotion'] == '혐오'), 'Emotion'] = 6 

In [16]:
data_list = []
for q, label in zip(data['Sentence'], data['Emotion']):
  data_tmp = []
  data_tmp.append(q)
  data_tmp.append(str(label))

  data_list.append(data_tmp)

In [17]:
data_list[:5]

[['언니 동생으로 부르는게 맞는 일인가요..??', '0'],
 ['그냥 내 느낌일뿐겠지?', '0'],
 ['아직너무초기라서 그런거죠?', '0'],
 ['유치원버스 사고 낫다던데', '0'],
 ['근데 원래이런거맞나요', '0']]

In [18]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list,
                                               test_size=0.25, random_state=0)

In [19]:
print(len(dataset_train))
print(len(dataset_test))

28945
9649


In [20]:
dataset_train[0]

['그냥 멋져유 ~요리할땐 더 멋져유~기부했다고 들었을땐 더더더 멋져부러유~~^^', '5']

In [21]:
# 데이터를 train과 test로 나누었다면 각 데이터가
# KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화
# 정수 인코딩, 패딩 들을 해주어야 한다.
# 예시 코드에 입력데이터의 형태가 되도록 해주는 클래스가 있어서
# 동일하게 코드를 작성해 주었다.
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,
               max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
    
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i],))

  def __len__(self):
    return (len(self.labels))

In [22]:
# Setting parameters
max_len =64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1 # 그래디언트 클리핑(https://wikidocs.net/61375)
log_interval = 200
learning_rate = 5e-5

In [23]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [24]:
data_train[0]

(array([   2, 1189,  517, 6188, 7245, 7063,  517,  463, 3486, 7836, 5966,
        1698,  517, 6188, 7245, 7063,  517,  463, 1281, 7870, 1801, 6885,
        7088, 5966, 1698, 5837, 5837,  517, 6188, 7245, 6398, 6037, 7063,
         517,  463,  517,  463,  517,  364,  517,  364,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(42, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 5)

- 3개의 array가 출력되는데, 첫 번째는 패딩된 시퀀스, 두번쨰는 길이와 타입에 대한 내용, 세번째는 어텐션 마스크 시퀀스

In [25]:
# torch 형식의 dataset을 생성
train_dataloader = torch.utils.data.DataLoader(data_train,
                                               batch_size=batch_size,
                                               num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test,
                                               batch_size=batch_size,
                                               num_workers=2)

In [26]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 7, # 클래스 수 조정
               dr_rate=None,
               params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gent_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gent_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids,
                          token_type_ids = segment_ids.long(),
                          attention_mask = attention_mask.float().to(token_ids.device))
    
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

In [27]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [28]:
for name, param in model.named_parameters():
  print(f'name:{name}')
  print(type(param))
  print(f'param.shape:{param.shape}')
  print(f'param.requires_grad:{param.requires_grad}')
  print('=====')

name:bert.embeddings.word_embeddings.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([8002, 768])
param.requires_grad:True
=====
name:bert.embeddings.position_embeddings.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([512, 768])
param.requires_grad:True
=====
name:bert.embeddings.token_type_embeddings.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([2, 768])
param.requires_grad:True
=====
name:bert.embeddings.LayerNorm.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([768])
param.requires_grad:True
=====
name:bert.embeddings.LayerNorm.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([768])
param.requires_grad:True
=====
name:bert.encoder.layer.0.attention.self.query.weight
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Size([768, 768])
param.requires_grad:True
=====
name:bert.encoder.layer.0.attention.self.query.bias
<class 'torch.nn.parameter.Parameter'>
param.shape:torch.Si

In [29]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_step,
                                            num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [30]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/453 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0013246536254883 train acc 0.1875
epoch 1 batch id 201 loss 1.4540374279022217 train acc 0.2562966417910448
epoch 1 batch id 401 loss 1.2743005752563477 train acc 0.35095854114713215
epoch 1 train acc 0.366303889105311


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 1 test acc 0.49883430193269357


  0%|          | 0/453 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5422628902554399


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.5140936374664307 train acc 0.78125
epoch 5 batch id 201 loss 0.5756090879440308 train acc 0.7568407960199005
epoch 5 batch id 401 loss 0.5520973801612854 train acc 0.7701059850374065
epoch 5 train acc 0.7722211401116738


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 5 test acc 0.5436798891742127


In [31]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):
  
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0,1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test,
                                                batch_size = batch_size,
                                                num_workers=5)
  
  model.eval()

  print('입력문장 : ', predict_sentence)
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long(). to(device)

    out = model(token_ids, valid_length, segment_ids)

    test_eval=[]
    for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()

      if np.argmax(logits) == 0:
        test_eval.append('공포가') 
      elif np.argmax(logits) == 1:
        test_eval.append('놀람이')
      elif np.argmax(logits) == 2:
        test_eval.append('분노가')
      elif np.argmax(logits) == 3:
        test_eval.append('슬픔이')
      elif np.argmax(logits) == 4:
        test_eval.append('중립적')
      elif np.argmax(logits) == 5:
        test_eval.append('행복이')
      elif np.argmax(logits) == 6:
        test_eval.append('혐오가')
  print(f'>> 입력한 문장은 {test_eval[0]} 느껴집니다. ')

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [34]:
predict('너무 슬퍼')

입력문장 :  너무 슬퍼
>> 입력한 문장은 슬픔이 느껴집니다. 
